# 해당 파일 설명
이 코드를 실행하기 위한 파일, /model 경로에 다음의 다섯가지 파일이 필요합니다.

1. model_tem_out_loc.pkl
2. model_hum_out_loc.pkl
3. model_tem_in.pkl
4. model_hum_in.pkl
5. model_tem_coil.pkl

## 추가 설명

sklearn을 제외하고 사용자 함수를 통해 모델의

Train MSE를 구합니다. 이를 통해 모델이 잘 적합됬는지 1차적으로 검증합니다.

# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
#BASE_WORKING_DIR = "/content/gdrive/My Drive/Colab Notebooks/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.csv
result.gsheet
test_cond.csv
train_ta_hm.csv

===== Convert

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Get the Data

## 1. plant data 가져오기

In [ ]:
# plant test 데이터 
data_train = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature_add_cluster.csv"))
data_test = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_test_all_weather_feature_add_cluster.csv"))

# unnamed: 0 제거
data_train.drop(['Unnamed: 0'], axis='columns', inplace=True)
data_test.drop(['Unnamed: 0'], axis='columns', inplace=True)

# date 변수 str -> datetime.date()
data_train['date'] = data_train['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 
data_test['date'] = data_test['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 

In [ ]:
data_test.isnull().sum()

mea_ddhr                 0
plant                    0
loc                      0
date                     0
time                     0
dangjin-ta               0
dangjin-hm               0
dangjin-ws               0
dangjin-rn               0
sinpyeong-ta             0
sinpyeong-ws             0
sinpyeong-rn             0
sinpyeong-lp             0
sinpyeong-ap             0
seosan-ta                0
seosan-hm                0
seosan-ws                0
seosan-rn                0
seosan-dew               0
seosan-sp                0
seosan-lp                0
seosan-ap                0
seosan-sr                0
seosan-ss                0
seosan-ground            0
songsan-ta               0
songsan-hm               0
songsan-ws               0
songsan-rn               0
songak-ta                0
songak-hm                0
songak-ws                0
songak-rn                0
seosan-ta-trend          0
seosan-hm-trend          0
seosan-ground-trend      0
seosan-ta-point          0
s

# 5. model 값과 real 값 비교하기 (검증)

In [ ]:
# tem_out_loc, hum_out_loc 모델
model_tem_out_loc = joblib.load(os.path.join(MODEL_PATH, "model_tem_out_loc.pkl"))
model_hum_out_loc = joblib.load(os.path.join(MODEL_PATH, "model_hum_out_loc.pkl"))

model_out_loc = [model_tem_out_loc, model_hum_out_loc]
model_out_loc_name = ['tem_out_loc', 'hum_out_loc']

In [ ]:
# 예측에 필요한 컬럼 정의
data_columns = list(data_test.columns)
remove_list = ['mea_ddhr', 'date']
for rmv in remove_list:
  data_columns.remove(rmv)
data_columns

['plant',
 'loc',
 'time',
 'dangjin-ta',
 'dangjin-hm',
 'dangjin-ws',
 'dangjin-rn',
 'sinpyeong-ta',
 'sinpyeong-ws',
 'sinpyeong-rn',
 'sinpyeong-lp',
 'sinpyeong-ap',
 'seosan-ta',
 'seosan-hm',
 'seosan-ws',
 'seosan-rn',
 'seosan-dew',
 'seosan-sp',
 'seosan-lp',
 'seosan-ap',
 'seosan-sr',
 'seosan-ss',
 'seosan-ground',
 'songsan-ta',
 'songsan-hm',
 'songsan-ws',
 'songsan-rn',
 'songak-ta',
 'songak-hm',
 'songak-ws',
 'songak-rn',
 'seosan-ta-trend',
 'seosan-hm-trend',
 'seosan-ground-trend',
 'seosan-ta-point',
 'seosan-hm-point',
 'seosan-ground-point',
 'seosan-ta-changed',
 'seosan-hm-changed',
 'seosan-ground-changed']

In [ ]:
for i in range(len(model_out_loc_name)):
  data_train[model_out_loc_name[i] + '_pred'] = model_out_loc[i].predict(data_train[data_columns])
  
data_train

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed,tem_out_loc_pred,hum_out_loc_pred
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,7.00,59.00
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,7.00,59.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.57,51.47
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.57,51.47
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.47,53.14
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.47,53.14


In [ ]:
error_tem_out = data_train['tem_out_loc'] -  data_train['tem_out_loc_pred']
error_tem_out = np.array(error_tem_out)
print("tem_out mean square error : ", np.matmul(error_tem_out, error_tem_out.T))

tem_out mean square error :  2.0329592293743844e-22


In [ ]:
error_hum_out = data_train['hum_out_loc'] -  data_train['hum_out_loc_pred']
error_hum_out = np.array(error_hum_out)
print("hum_out mean square error : ", np.matmul(error_hum_out, error_hum_out.T))

hum_out mean square error :  9.491999999844058e-05


In [ ]:
# tem_out_loc, hum_out_loc 모델
model_tem_in = joblib.load(os.path.join(MODEL_PATH, "model_tem_in.pkl"))
model_hum_in = joblib.load(os.path.join(MODEL_PATH, "model_hum_in.pkl"))
model_tem_coil = joblib.load(os.path.join(MODEL_PATH, "model_tem_coil.pkl"))


model_in = [model_tem_in, model_hum_in, model_tem_coil]
model_in_name = ['tem_in', 'hum_in', 'tem_coil']

In [ ]:
for i in range(len(model_in_name)):
  data_train[model_in_name[i] + '_pred'] = model_in[i].predict(data_train[data_columns])
  
data_train

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed,tem_out_loc_pred,hum_out_loc_pred,tem_in_pred,hum_in_pred,tem_coil_pred
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00,16.00,24.00,11.00
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00,14.00,23.00,11.00
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,9.00,42.00,13.00,32.00,10.00
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,7.00,59.00,14.00,28.00,10.00
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,3,2,1,2,0,2,18.696296,56.993994,44.577377,7.00,59.00,12.00,32.00,9.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.57,51.47,9.05,38.13,9.76
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.57,51.47,8.32,39.13,9.49
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,0,1,2,3,4,1,5.397598,46.997998,17.498498,4.47,53.14,8.99,37.33,9.15
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.6

In [ ]:
error_tem_in = data_train['tem_in'] -  data_train['tem_in_pred']
error_tem_in = np.array(error_tem_in)
print("tem_in mean square error : ", np.matmul(error_tem_in, error_tem_in.T))

tem_in mean square error :  2.392614719581556e-22


In [ ]:
error_hum_in = data_train['hum_in'] -  data_train['hum_in_pred']
error_hum_in = np.array(error_hum_in)
print("hum_in mean square error : ", np.matmul(error_hum_in, error_hum_in.T))

hum_in mean square error :  0.45945537999999486


In [ ]:
error_tem_coil = data_train['tem_coil'] -  data_train['tem_coil_pred']
error_tem_coil = np.array(error_tem_coil)
print("tem_coil mean square error : ", np.matmul(error_tem_coil, error_tem_coil.T))

tem_coil mean square error :  0.00080016000000013
